<a href="https://colab.research.google.com/github/sinchana-08/AI/blob/main/1BM22CS278_ResolutioninFOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Tree-like structure where each node represents a state
tree = {
    'A': {'B': {'D': 3, 'E': 12}, 'C': {'F': 8, 'G': 2}}
}

def terminal_test(state):
    """Returns True if the state is a terminal node."""
    return isinstance(state, int)  # Terminal states are integers (utility values)

def utility(state):
    """Returns the utility value of a terminal state."""
    return state  # State itself is the utility value (integer)

def actions(state):
    """Returns the possible actions (keys of the current state's dictionary)."""
    return list(state.keys())

def result(state, action):
    """Returns the child state resulting from taking the given action."""
    return state[action]

def alpha_beta_search(state):
    """Alpha-Beta Pruning to find the maximum number."""
    def max_value(state, alpha, beta):
        if terminal_test(state):
            return utility(state)
        v = float('-inf')
        for action in actions(state):
            v = max(v, min_value(result(state, action), alpha, beta))
            if v >= beta:
                return v  # Beta cutoff
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if terminal_test(state):
            return utility(state)
        v = float('inf')
        for action in actions(state):
            v = min(v, max_value(result(state, action), alpha, beta))
            if v <= alpha:
                return v  # Alpha cutoff
            beta = min(beta, v)
        return v

    # Start with initial alpha and beta values
    return max_value(state, float('-inf'), float('inf'))

# Run Alpha-Beta Pruning to find the maximum number in the tree
max_value_found = alpha_beta_search(tree['A'])
print(f"Maximum value in the tree: {max_value_found}")


Maximum value in the tree: 3


In [9]:
class CNFReasoner:
    def __init__(self, clauses):
        """
        Initializes the CNF Reasoner.

        Parameters:
            clauses (list of sets): List of clauses in CNF format.
        """
        self.clauses = [set(clause) for clause in clauses]

    def resolve(self, clause1, clause2):
        """
        Resolve two clauses to produce new clauses if possible.

        Parameters:
            clause1 (set): The first clause (set of literals).
            clause2 (set): The second clause (set of literals).

        Returns:
            list: A list of resolved clauses (sets of literals).
        """
        resolvents = []
        for literal in clause1:
            neg_literal = f"~{literal}" if not literal.startswith("~") else literal[1:]
            if neg_literal in clause2:
                # Create a new clause by removing complementary literals
                new_clause = (clause1 - {literal}) | (clause2 - {neg_literal})
                resolvents.append(new_clause)
        return resolvents

    def infer(self, goal):
        """
        Infer whether the goal is provable using resolution.

        Parameters:
            goal (set): The negation of the goal to be proved.

        Returns:
            bool: True if the goal is provable, False otherwise.
        """
        goal_clause = {f"~{literal}" for literal in goal}
        clauses = self.clauses + [goal_clause]  # Add the negated goal to clauses
        new = set()

        while True:
            pairs = [(clauses[i], clauses[j]) for i in range(len(clauses)) for j in range(i + 1, len(clauses))]
            for (clause1, clause2) in pairs:
                resolvents = self.resolve(clause1, clause2)
                for resolvent in resolvents:
                    if not resolvent:  # Found an empty clause, goal is proved
                        return True
                    new.add(frozenset(resolvent))

            # Check if no new information is being added
            if new.issubset(set(map(frozenset, clauses))):
                return False

            # Add new resolvents to the clauses
            for clause in new:
                if clause not in clauses:
                    clauses.append(set(clause))


# Define the CNF clauses based on the FOL premises
cnf_clauses = [
    {"~Food(X)", "Likes(John, X)"},  # Rule 1: If Food(X), then Likes(John, X)
    {"~Eats(Anil, Peanuts)", "Food(Peanuts)"},  # Rule 2.1: If Anil eats peanuts, peanuts are food
    {"~Alive(Anil)", "Food(Peanuts)"},  # Rule 2.2: If Anil is alive, peanuts are food
    {"~Food(Peanuts)", "Likes(John, Peanuts)"},  # Rule 3: If peanuts are food, John likes peanuts
    {"Eats(Anil, Peanuts)"},  # Fact 1: Anil eats peanuts
    {"Alive(Anil)"},  # Fact 2: Anil is alive
]

# Define the goal: Prove that John likes peanuts
goal = {"Likes(John, Peanuts)"}

# Initialize the reasoner and infer
reasoner = CNFReasoner(cnf_clauses)
if reasoner.infer(goal):
    print("Proved: John likes peanuts.")
else:
    print("Could not prove: John likes peanuts.")



Proved: John likes peanuts.
